<a href="https://www.kaggle.com/code/tunguz/toxic-comments-with-embeddings-and-rapids-svc?scriptVersionId=108573173" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cuml, cudf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_large_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_large_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_large_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_large_320.npy


In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from cuml.svm import SVC
from sklearn.model_selection import KFold
from scipy.sparse import hstack
from sklearn.metrics import roc_auc_score

In [3]:
%%time
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

target = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip').fillna(' ')[class_names].values
target

CPU times: user 1.21 s, sys: 107 ms, total: 1.32 s
Wall time: 1.9 s


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [4]:
train_features = np.load('../input/toxic-comments-transformer-embeddings/train_embs_large_400.npy')
test_features = np.load('../input/toxic-comments-transformer-embeddings/test_embs_large_400.npy')

In [5]:
%%time
train_oof = np.zeros(target.shape)
test_preds = np.zeros((test_features.shape[0],6))
N_FOLDS = 3
kf = KFold(n_splits=N_FOLDS, random_state=137, shuffle=True)
for ii in range(6):
    print("Fitting target", ii+1)
    for jj, (train_index, val_index) in enumerate(kf.split(train_features)):
        print("Fitting fold", jj+1)
        train_x = train_features[train_index]
        val_x = train_features[val_index]
        train_target = target[train_index,ii]
        classifier = SVC(C=10, probability=True)
        classifier.fit(train_x, train_target)
        train_oof[val_index, ii] = classifier.predict_proba(val_x)[:,1]
        test_preds[:, ii] += classifier.predict_proba(test_features)[:,1]/N_FOLDS
    print(roc_auc_score(target[:,ii], train_oof[:,ii]))

Fitting target 1
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9745359646788082
Fitting target 2
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9678934068474033
Fitting target 3
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9804009462097619
Fitting target 4
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9689023501345637
Fitting target 5
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.9761707518606775
Fitting target 6
Fitting fold 1
Fitting fold 2
Fitting fold 3
0.957335110735273
CPU times: user 4min 47s, sys: 38.1 s, total: 5min 25s
Wall time: 5min


In [6]:
test_preds

array([[9.94145513e-01, 1.80781480e-02, 9.81910781e-01, 1.44500676e-02,
        9.53989490e-01, 2.27201348e-01],
       [8.35708121e-03, 7.95194447e-04, 1.82808634e-03, 8.60258056e-05,
        9.70881946e-04, 5.24757071e-04],
       [1.40858835e-02, 1.64338296e-03, 8.67102485e-04, 3.36380227e-05,
        7.24877846e-04, 5.99358976e-04],
       ...,
       [3.25784766e-03, 1.94947609e-03, 1.84186834e-03, 1.00490422e-04,
        1.72034742e-03, 4.57444508e-03],
       [1.75788814e-02, 7.60537865e-03, 2.31202582e-02, 2.80751976e-03,
        9.70323735e-03, 1.20035441e-02],
       [7.52856326e-01, 1.32268858e-02, 2.08003147e-01, 3.63898069e-03,
        2.62592397e-01, 3.49339730e-03]])

In [7]:
submission = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5


In [8]:
submission[class_names] = test_preds
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.994146,0.018078,0.981911,0.014450,0.953989,0.227201
1,0000247867823ef7,0.008357,0.000795,0.001828,0.000086,0.000971,0.000525
2,00013b17ad220c46,0.014086,0.001643,0.000867,0.000034,0.000725,0.000599
3,00017563c3f7919a,0.001090,0.002443,0.002300,0.000063,0.001763,0.000735
4,00017695ad8997eb,0.005919,0.001401,0.001838,0.001330,0.001438,0.001554


In [9]:
submission.to_csv('submission.csv', index=False)